# Basic Comparison Graphs

In [2]:
# Import Libraries and Dependencies 
import pandas as pd
import sqlite3
import sys
import requests
from ipywidgets import interact, widgets, fixed
from IPython.display import display
import altair as alt


In [3]:
# Connect to SQLite database
conn = sqlite3.connect('Data/menu.db')

# Query to retrieve data from the 'menu' table
query = 'SELECT * FROM menu'

# Use Pandas to read data from SQLite into a DataFrame
menu_df = pd.read_sql(query, conn)

# Close the connection
conn.close()

# Display the DataFrame
menu_df.head(2)

,restaurant,item,item_with_chain,calories,cal_fat,total_fat,sat_fat,trans_fat,cholesterol,sodium,total_carb,fiber,sugar,protein
0,Mcdonalds,Artisan Grilled Chicken Sandwich,Artisan Grilled Chicken Sandwich (Mcdonalds),380.0,60.0,7.0,2.0,0.0,95.0,1110.0,44.0,3.0,11.0,37.0
1,Mcdonalds,Single Bacon Smokehouse Burger,Single Bacon Smokehouse Burger (Mcdonalds),840.0,410.0,45.0,17.0,1.5,130.0,1580.0,62.0,2.0,18.0,46.0


In [4]:


# Enable notebook rendering for Altair
alt.renderers.enable('default')

# Define a function to display information for the selected items
def display_items_info(selected_item1, selected_item2):
    item_data1 = menu_df[menu_df['item_with_chain'] == selected_item1].squeeze()
    item_data2 = menu_df[menu_df['item_with_chain'] == selected_item2].squeeze()
    
    # Update the keys with the string values of the selected items
    comparison_df = pd.concat([item_data1, item_data2], axis=1, keys=[selected_item1, selected_item2])

    # Melt the DataFrame for Altair
    melted_df = comparison_df.reset_index().melt(id_vars=['index'], var_name='Item', value_name='Value')

    # Adjust the chart to display individual bars side by side
    chart = alt.Chart(melted_df).mark_bar().encode(
        x=alt.X('index:N', title=' '),
        y=alt.Y('Value:Q', title=' '),
        color=alt.Color('Item:N', scale=alt.Scale(range=['#6495ED', '#ADD8E6']), legend=None),
        tooltip=['Item:N', alt.Tooltip('Value:Q', title='Value')],
    ).properties(
        width=300,
        title=" "
    ).facet(
        column='Item:N',
        title=alt.TitleParams(text="Comparing the Nutrients of the Two Selected Items", offset=10, orient='top')
    )

    display(chart)



# Create two dropdown widgets with the list of items
item_dropdown1 = widgets.Dropdown(options=menu_df['item_with_chain'].unique(), description='Select Item 1:')
item_dropdown2 = widgets.Dropdown(options=menu_df['item_with_chain'].unique(), description='Select Item 2:', value=menu_df['item_with_chain'].unique()[1])

# Connect the dropdown widgets to the display_items_info function
interact(display_items_info, selected_item1=item_dropdown1, selected_item2=item_dropdown2)


interactive(children=(Dropdown(description='Select Item 1:', options=('Artisan Grilled Chicken Sandwich (Mcdon…

<function __main__.display_items_info(selected_item1, selected_item2)>

In [5]:
import pandas as pd
import altair as alt
from IPython.display import display
from ipywidgets import interact, widgets, fixed

# Enable notebook rendering for Altair
alt.renderers.enable('default')

# Define a function to display information for the selected items
def display_items_info(selected_item1, selected_item2, selected_column):
    item_data1 = menu_df[menu_df['item_with_chain'] == selected_item1].squeeze()
    item_data2 = menu_df[menu_df['item_with_chain'] == selected_item2].squeeze()

    # Filter only the selected column for both items
    item_data1 = item_data1[[selected_column]]
    item_data2 = item_data2[[selected_column]]

    # Update the keys with the string values of the selected items
    comparison_df = pd.concat([item_data1, item_data2], axis=1, keys=[selected_item1, selected_item2])

    # Melt the DataFrame for Altair
    melted_df = comparison_df.reset_index().melt(id_vars=['index'], var_name='Item', value_name='Value')

    # Adjust the chart to display individual bars side by side
    chart = alt.Chart(melted_df).mark_bar().encode(
        x=alt.X('index:N', title=' '),
        y=alt.Y('Value:Q', title=' '),
        color=alt.Color('Item:N', scale=alt.Scale(range=['#6495ED', '#ADD8E6']), legend=None),
        tooltip=['Item:N', alt.Tooltip('Value:Q', title='Value')],
    ).properties(
        width=300,
        title=" "
    ).facet(
        column='Item:N',
        title=alt.TitleParams(text=f"Comparing {selected_column} of the Two Selected Items", offset=10, orient='top')
    )

    display(chart)

# Create dropdown widgets for items and columns
item_dropdown1 = widgets.Dropdown(options=menu_df['item_with_chain'].unique(), description='Select Item 1:')
item_dropdown2 = widgets.Dropdown(options=menu_df['item_with_chain'].unique(), description='Select Item 2:', value=menu_df['item_with_chain'].unique()[1])
column_dropdown = widgets.Dropdown(options=menu_df.columns[3:].tolist(), description='Select Column:', value='calories')

# Connect the dropdown widgets to the display_items_info function
interact(display_items_info, selected_item1=item_dropdown1, selected_item2=item_dropdown2, selected_column=column_dropdown)


interactive(children=(Dropdown(description='Select Item 1:', options=('Artisan Grilled Chicken Sandwich (Mcdon…

<function __main__.display_items_info(selected_item1, selected_item2, selected_column)>

In [6]:
# Enable notebook rendering for Altair
alt.renderers.enable('default')

# Assume 'menu_df' is your DataFrame containing the nutrition data
# You can load your data using: menu_df = pd.read_csv('your_data.csv')

# Define a function to display information for the selected restaurants
def display_restaurants_info(selected_restaurant1, selected_restaurant2, selected_column):
    restaurant_data1 = menu_df[menu_df['restaurant'] == selected_restaurant1]
    restaurant_data2 = menu_df[menu_df['restaurant'] == selected_restaurant2]

    # Calculate mean values for the selected column for both restaurants
    mean_values1 = restaurant_data1[selected_column].mean()
    mean_values2 = restaurant_data2[selected_column].mean()

    # Create a DataFrame with mean values
    comparison_df = pd.DataFrame({
        selected_restaurant1: [mean_values1],
        selected_restaurant2: [mean_values2]
    })

    # Melt the DataFrame for Altair
    melted_df = comparison_df.reset_index().melt(id_vars=['index'], var_name='Restaurant', value_name='Mean Value')

    # Create the chart
    chart = alt.Chart(melted_df).mark_bar().encode(
        x=alt.X('index:N', title=' '),
        y=alt.Y('Mean Value:Q', title=' '),
        color=alt.Color('Restaurant:N', scale=alt.Scale(range=['#6495ED', '#ADD8E6']), legend=None),
        tooltip=['Restaurant:N', alt.Tooltip('Mean Value:Q', title='Mean Value')],
    ).properties(
        width=300,
        title=" "
    ).facet(
        column='Restaurant:N',
        title=alt.TitleParams(text=f"Comparing Mean {selected_column} of the Two Selected Restaurants", offset=10, orient='top')
    )

    display(chart)

# Create dropdown widgets for restaurants and columns
restaurant_dropdown1 = widgets.Dropdown(options=menu_df['restaurant'].unique(), description='Select Restaurant 1:')
restaurant_dropdown2 = widgets.Dropdown(options=menu_df['restaurant'].unique(), description='Select Restaurant 2:', value=menu_df['restaurant'].unique()[1])
column_dropdown = widgets.Dropdown(options=menu_df.columns[3:].tolist(), description='Select Column:', value='calories')

# Connect the dropdown widgets to the display_restaurants_info function
interact(display_restaurants_info, selected_restaurant1=restaurant_dropdown1, selected_restaurant2=restaurant_dropdown2, selected_column=column_dropdown)

interactive(children=(Dropdown(description='Select Restaurant 1:', options=('Mcdonalds', 'Chick Fil-A', 'Sonic…

<function __main__.display_restaurants_info(selected_restaurant1, selected_restaurant2, selected_column)>

In [7]:
summary_table1 = menu_df.groupby('restaurant').mean(numeric_only=True)
summary_table1

,calories,cal_fat,total_fat,sat_fat,trans_fat,cholesterol,sodium,total_carb,fiber,sugar,protein
restaurant,,,,,,,,,,,
Arbys,532.727273,237.836364,26.981818,7.972727,0.418182,70.454545,1515.272727,44.872727,2.709091,7.563636,29.254545
Burger King,608.571429,333.757143,36.814286,11.150000,0.864286,100.857143,1223.571429,39.314286,2.633882,8.185714,29.984158
Chick Fil-A,384.444444,145.370370,16.148148,4.111111,0.037037,79.074074,1151.481481,28.629630,2.454606,4.148148,31.703704
Dairy Queen,520.238095,260.476190,28.857143,10.440476,0.678571,71.547619,1181.785714,38.690476,2.833333,6.357143,24.833333
Mcdonalds,640.350877,285.614035,31.807018,8.289474,0.464912,109.736842,1437.894737,48.789474,3.228070,11.070175,40.298246
Sonic,631.698113,338.301887,37.641509,11.415094,0.933962,86.981132,1350.754717,47.207547,2.660377,6.528302,29.188679
Subway,503.020833,165.104167,18.479167,6.197917,0.218750,61.302083,1272.968750,54.718750,6.562500,10.093750,30.312500
Taco Bell,443.008850,187.699115,20.858407,6.557522,0.243363,38.893805,1012.389381,46.575221,5.699115,3.690265,17.380531


In [19]:
# Initialize an empty list to store DataFrames
dfs = []

# Loop through each nutrient column
for nutrient in menu_df.columns[3:]:  # Assuming nutrient columns start from index 3
    # Find the row index of the item with the minimum value for the current nutrient
    min_idx = menu_df[nutrient].idxmin()
    # Find the row index of the item with the maximum value for the current nutrient
    max_idx = menu_df[nutrient].idxmax()
    
    # Get the 'item_with_chain' value for the item with the minimum value
    min_item_with_chain = menu_df.loc[min_idx, 'item_with_chain']
    # Get the 'item_with_chain' value for the item with the maximum value
    max_item_with_chain = menu_df.loc[max_idx, 'item_with_chain']
    
    # Get the minimum and maximum values for the current nutrient
    min_value = menu_df.loc[min_idx, nutrient]
    max_value = menu_df.loc[max_idx, nutrient]
    
    # Create a DataFrame for the current nutrient
    df = pd.DataFrame({
        'nutrient': [nutrient],
        'min_item_with_chain': [min_item_with_chain],
        'min_value': [min_value],
        'max_item_with_chain': [max_item_with_chain],
        'max_value': [max_value]
    })
    
    # Append the DataFrame to the list
    dfs.append(df)

# Concatenate all DataFrames in the list
summary_table2 = pd.concat(dfs, ignore_index=True)

# Display the summary table
summary_table2

,nutrient,min_item_with_chain,min_value,max_item_with_chain,max_value
0,calories,Side Salad (Dairy Queen),20.0,20 piece Buttermilk Crispy Chicken Tenders (Mc...,2430.0
1,cal_fat,Side Salad (Dairy Queen),0.0,20 piece Buttermilk Crispy Chicken Tenders (Mc...,1270.0
2,total_fat,Side Salad (Dairy Queen),0.0,20 piece Buttermilk Crispy Chicken Tenders (Mc...,141.0
3,sat_fat,Smokehouse BBQ Bacon Sandwich (Chick Fil-A),0.0,American Brewhouse King (Burger King),47.0
4,trans_fat,Artisan Grilled Chicken Sandwich (Mcdonalds),0.0,American Brewhouse King (Burger King),8.0
5,cholesterol,Premium Asian Salad w/o Chicken (Mcdonalds),0.0,American Brewhouse King (Burger King),805.0
6,sodium,Side Salad (Dairy Queen),15.0,20 piece Buttermilk Crispy Chicken Tenders (Mc...,6080.0
7,total_carb,Regular Cheese Curds (Dairy Queen),0.0,10 piece Sweet N' Spicy Honey BBQ Glazed Tende...,156.0
8,fiber,3 piece Buttermilk Crispy Chicken Tenders (Mcd...,0.0,Cantina Power Burrito - Veggie (Taco Bell),17.0
9,sugar,3 piece Buttermilk Crispy Chicken Tenders (Mcd...,0.0,10 piece Sweet N' Spicy Honey BBQ Glazed Tende...,87.0


In [20]:
top_protein_items = menu_df.sort_values(by='protein', ascending=False).head(10)
top_protein_items

,restaurant,item,item_with_chain,calories,cal_fat,total_fat,sat_fat,trans_fat,cholesterol,sodium,total_carb,fiber,sugar,protein
39,Mcdonalds,20 piece Buttermilk Crispy Chicken Tenders,20 piece Buttermilk Crispy Chicken Tenders (Mc...,2430.0,1270.0,141.0,24.0,2.0,475.0,6080.0,103.0,2.0,3.0,186.0
192,Burger King,American Brewhouse King,American Brewhouse King (Burger King),1550.0,1134.0,126.0,47.0,8.0,805.0,1820.0,21.0,3.0,7.0,134.0
38,Mcdonalds,12 piece Buttermilk Crispy Chicken Tenders,12 piece Buttermilk Crispy Chicken Tenders (Mc...,1510.0,790.0,88.0,15.0,1.0,295.0,3770.0,64.0,1.0,2.0,115.0
69,Chick Fil-A,30 piece Chicken Nuggets,30 piece Chicken Nuggets (Chick Fil-A),970.0,414.0,46.0,2.5,0.0,285.0,3660.0,35.0,4.0,1.0,103.0
44,Mcdonalds,40 piece Chicken McNuggets,40 piece Chicken McNuggets (Mcdonalds),1770.0,960.0,107.0,18.0,0.5,295.0,3370.0,105.0,7.0,1.0,98.0
47,Mcdonalds,10 piece Sweet N' Spicy Honey BBQ Glazed Tenders,10 piece Sweet N' Spicy Honey BBQ Glazed Tende...,1600.0,600.0,66.0,10.0,0.0,265.0,4450.0,156.0,5.0,87.0,97.0
37,Mcdonalds,10 piece Buttermilk Crispy Chicken Tenders,10 piece Buttermilk Crispy Chicken Tenders (Mc...,1210.0,630.0,70.0,12.0,1.0,240.0,3230.0,52.0,1.0,4.0,94.0
326,Subway,Footlong Corned Beef Reuben,Footlong Corned Beef Reuben (Subway),940.0,260.0,30.0,9.0,0.0,170.0,3540.0,90.0,14.0,24.0,78.0
311,Subway,Footlong Big Philly Cheesesteak,Footlong Big Philly Cheesesteak (Subway),1000.0,300.0,34.0,18.0,2.0,170.0,2620.0,102.0,12.0,16.0,76.0
2,Mcdonalds,Double Bacon Smokehouse Burger,Double Bacon Smokehouse Burger (Mcdonalds),1130.0,600.0,67.0,27.0,3.0,220.0,1920.0,63.0,3.0,18.0,70.0


In [21]:
top_fiber_items = menu_df.sort_values(by='fiber', ascending=False).head(10)
top_fiber_items

,restaurant,item,item_with_chain,calories,cal_fat,total_fat,sat_fat,trans_fat,cholesterol,sodium,total_carb,fiber,sugar,protein
412,Taco Bell,Cantina Power Burrito - Veggie,Cantina Power Burrito - Veggie (Taco Bell),740.0,230.0,26.0,5.0,0.0,10.0,1750.0,107.0,17.0,8.0,20.0
367,Subway,Footlong Veggie Patty,Footlong Veggie Patty (Subway),780.0,140.0,14.0,2.0,0.0,20.0,1600.0,112.0,16.0,16.0,46.0
355,Subway,Footlong Turkey & Bacon Avocado,Footlong Turkey & Bacon Avocado (Subway),780.0,220.0,26.0,7.0,0.0,60.0,1720.0,98.0,16.0,14.0,44.0
332,Subway,Footlong Meatball Marinara,Footlong Meatball Marinara (Subway),960.0,320.0,36.0,14.0,2.0,60.0,1900.0,118.0,16.0,24.0,42.0
81,Chick Fil-A,Chargrilled Chicken Cool Wrap,Chargrilled Chicken Cool Wrap (Chick Fil-A),350.0,126.0,14.0,5.0,0.0,60.0,960.0,29.0,15.0,3.0,37.0
326,Subway,Footlong Corned Beef Reuben,Footlong Corned Beef Reuben (Subway),940.0,260.0,30.0,9.0,0.0,170.0,3540.0,90.0,14.0,24.0,78.0
487,Taco Bell,Cantina Power Bowl - Veggie,Cantina Power Bowl - Veggie (Taco Bell),540.0,190.0,21.0,3.0,0.0,10.0,1310.0,75.0,14.0,4.0,14.0
477,Taco Bell,Nachos BellGrande®,Nachos BellGrande® (Taco Bell),760.0,360.0,39.0,6.0,0.0,30.0,1100.0,82.0,13.0,5.0,18.0
411,Taco Bell,Cantina Power Burrito - Steak,Cantina Power Burrito - Steak (Taco Bell),780.0,250.0,28.0,7.0,0.0,50.0,1900.0,98.0,13.0,7.0,33.0
430,Taco Bell,XXL Grilled Stuft Burrito - Beef,XXL Grilled Stuft Burrito - Beef (Taco Bell),880.0,380.0,42.0,14.0,1.0,75.0,2020.0,94.0,12.0,6.0,31.0


In [22]:
top_carb_items = menu_df.sort_values(by='total_carb', ascending=False).head(10)
top_carb_items

,restaurant,item,item_with_chain,calories,cal_fat,total_fat,sat_fat,trans_fat,cholesterol,sodium,total_carb,fiber,sugar,protein
47,Mcdonalds,10 piece Sweet N' Spicy Honey BBQ Glazed Tenders,10 piece Sweet N' Spicy Honey BBQ Glazed Tende...,1600.0,600.0,66.0,10.0,0.0,265.0,4450.0,156.0,5.0,87.0,97.0
122,Sonic,5 Piece Super Crunch Chicken Strip Dinner,5 Piece Super Crunch Chicken Strip Dinner (Sonic),1190.0,510.0,57.0,10.0,1.0,90.0,2610.0,126.0,8.0,9.0,44.0
270,Dairy Queen,6 Piece Chicken Strip Basket w/ Country Gravy,6 Piece Chicken Strip Basket w/ Country Gravy ...,1260.0,590.0,66.0,11.0,1.0,120.0,3500.0,121.0,12.0,4.0,49.0
351,Subway,Footlong Sweet Onion Chicken Teriyaki,Footlong Sweet Onion Chicken Teriyaki (Subway),760.0,80.0,10.0,2.0,0.0,100.0,1800.0,118.0,10.0,36.0,52.0
332,Subway,Footlong Meatball Marinara,Footlong Meatball Marinara (Subway),960.0,320.0,36.0,14.0,2.0,60.0,1900.0,118.0,16.0,24.0,42.0
121,Sonic,4 Piece Super Crunch Chicken Strip Dinner,4 Piece Super Crunch Chicken Strip Dinner (Sonic),1080.0,460.0,51.0,9.0,1.0,75.0,2390.0,118.0,8.0,9.0,37.0
367,Subway,Footlong Veggie Patty,Footlong Veggie Patty (Subway),780.0,140.0,14.0,2.0,0.0,20.0,1600.0,112.0,16.0,16.0,46.0
120,Sonic,3 Piece Super Crunch Chicken Strip Dinner,3 Piece Super Crunch Chicken Strip Dinner (Sonic),970.0,410.0,46.0,8.0,1.0,55.0,2160.0,109.0,7.0,9.0,30.0
412,Taco Bell,Cantina Power Burrito - Veggie,Cantina Power Burrito - Veggie (Taco Bell),740.0,230.0,26.0,5.0,0.0,10.0,1750.0,107.0,17.0,8.0,20.0
44,Mcdonalds,40 piece Chicken McNuggets,40 piece Chicken McNuggets (Mcdonalds),1770.0,960.0,107.0,18.0,0.5,295.0,3370.0,105.0,7.0,1.0,98.0


In [26]:
# Drop non-numeric columns
numeric_menu_df = menu_df.drop(columns=['restaurant', 'item', 'item_with_chain'])

# Generate summary statistics for each nutrient column
nutrient_statistics = numeric_menu_df.agg(['count', 'mean', 'median', 'min', 'max', 'std'])

# Display the summary statistics
nutrient_statistics

,calories,cal_fat,total_fat,sat_fat,trans_fat,cholesterol,sodium,total_carb,fiber,sugar,protein
count,513.000000,513.000000,513.000000,513.000000,513.000000,513.000000,513.000000,513.000000,513.000000,513.000000,513.000000
mean,531.111111,238.945419,26.604288,8.152047,0.462963,72.553606,1247.309942,45.647173,4.127965,7.272904,27.923764
median,490.000000,210.000000,23.000000,7.000000,0.000000,60.000000,1110.000000,44.000000,3.000000,6.000000,25.000000
min,20.000000,0.000000,0.000000,0.000000,0.000000,0.000000,15.000000,0.000000,0.000000,0.000000,1.000000
max,2430.000000,1270.000000,141.000000,47.000000,8.000000,805.000000,6080.000000,156.000000,17.000000,87.000000,186.000000
std,282.900251,166.683320,18.442620,6.431223,0.840615,63.259522,691.157795,24.924070,3.002546,6.770483,17.692028


In [27]:
filtered_items = menu_df[(menu_df['protein'] >= 30) & (menu_df['fiber'] >= 5) & (menu_df['total_fat'] <= 10)]

# Display the filtered items
filtered_items

,restaurant,item,item_with_chain,calories,cal_fat,total_fat,sat_fat,trans_fat,cholesterol,sodium,total_carb,fiber,sugar,protein
49,Mcdonalds,Premium Asian Salad w/ Grilled Chicken,Premium Asian Salad w/ Grilled Chicken (Mcdona...,270.0,80.0,9.0,1.0,0.0,80.0,740.0,18.0,5.0,10.0,31.0
314,Subway,Footlong Black Forest Ham,Footlong Black Forest Ham (Subway),580.0,80.0,10.0,2.0,0.0,40.0,1660.0,92.0,10.0,16.0,36.0
316,Subway,Footlong Carved Turkey,Footlong Carved Turkey (Subway),660.0,90.0,10.0,2.0,0.0,90.0,1780.0,90.0,10.0,14.0,50.0
334,Subway,Footlong Oven Roasted Chicken,Footlong Oven Roasted Chicken (Subway),640.0,80.0,10.0,4.0,0.0,50.0,1280.0,44.0,10.0,16.0,46.0
337,Subway,Footlong Roast Beef,Footlong Roast Beef (Subway),640.0,80.0,10.0,4.0,0.0,80.0,1400.0,90.0,10.0,14.0,48.0
345,Subway,Footlong Subway Club,Footlong Subway Club (Subway),620.0,80.0,10.0,4.0,0.0,80.0,1760.0,92.0,10.0,14.0,46.0
351,Subway,Footlong Sweet Onion Chicken Teriyaki,Footlong Sweet Onion Chicken Teriyaki (Subway),760.0,80.0,10.0,2.0,0.0,100.0,1800.0,118.0,10.0,36.0,52.0
358,Subway,Footlong Turkey Breast,Footlong Turkey Breast (Subway),560.0,60.0,8.0,2.0,0.0,40.0,1620.0,92.0,10.0,14.0,36.0
360,Subway,Footlong Turkey Breast & Ham,Footlong Turkey Breast & Ham (Subway),560.0,70.0,8.0,2.0,0.0,40.0,1640.0,92.0,10.0,16.0,36.0
